In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
import os
from scipy.io.matlab.mio import loadmat, savemat
import numpy as np
from PIL import Image
import torchvision
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch import nn, optim
import torch.nn.functional as nnf
import shutil
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

C:\Users\Nassim\AppData\Local\Temp\ipykernel_14256\1160388593.py:8: DeprecationWarning: Please use `loadmat` from the `scipy.io.matlab` namespace, the `scipy.io.matlab.mio` namespace is deprecated.
  from scipy.io.matlab.mio import loadmat, savemat
C:\Users\Nassim\AppData\Local\Temp\ipykernel_14256\1160388593.py:8: DeprecationWarning: Please use `savemat` from the `scipy.io.matlab` namespace, the `scipy.io.matlab.mio` namespace is deprecated.
  from scipy.io.matlab.mio import loadmat, savemat


In [3]:
from torchvision.models import resnet50, ResNet50_Weights

weights = ResNet50_Weights.DEFAULT
preprocess = weights.transforms()


In [4]:
model = resnet50(weights=ResNet50_Weights.DEFAULT)
num_features = model.fc.in_features
num_classes = 8
model.fc = nn.Sequential(
    nn.Linear(num_features, 1000),  # New layer with 1000 outputs (matches pre-trained model)
    nn.ReLU(),                       # Activation function (you can use others if needed)
    nn.Linear(1000, num_classes)     # Final layer with 8 outputs for your classification task
)

In [5]:
model = model.cuda()

In [2]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

Using cuda device


# Retraining RESNET on player data

In [12]:
path_pair_img = '../data/photoshoptest'
dataset = datasets.ImageFolder(root=path_pair_img, transform=preprocess)

batch_size = 5
train_dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

NameError: name 'model' is not defined

In [16]:
epochs = 20
torch.cuda.empty_cache()
for i in range(1):
    for t in range(epochs):
        print(f"Epoch {t+1}\n-------------------------------")
        train(train_dataloader, model, loss_fn, optimizer)
    print("Done!")


Epoch 1
-------------------------------
loss: 0.945551  [    5/ 6576]
loss: 0.984768  [  505/ 6576]
loss: 0.999304  [ 1005/ 6576]
loss: 0.952106  [ 1505/ 6576]
loss: 1.335849  [ 2005/ 6576]
loss: 0.821348  [ 2505/ 6576]
loss: 1.057048  [ 3005/ 6576]
loss: 1.318319  [ 3505/ 6576]
loss: 1.251393  [ 4005/ 6576]
loss: 1.014067  [ 4505/ 6576]
loss: 1.214785  [ 5005/ 6576]
loss: 0.857582  [ 5505/ 6576]
loss: 1.004859  [ 6005/ 6576]
loss: 0.619097  [ 6505/ 6576]
Epoch 2
-------------------------------
loss: 0.213788  [    5/ 6576]
loss: 0.997163  [  505/ 6576]
loss: 1.314827  [ 1005/ 6576]
loss: 0.931822  [ 1505/ 6576]
loss: 0.688763  [ 2005/ 6576]
loss: 0.623131  [ 2505/ 6576]
loss: 0.699024  [ 3005/ 6576]
loss: 0.651918  [ 3505/ 6576]
loss: 0.249230  [ 4005/ 6576]
loss: 0.698804  [ 4505/ 6576]
loss: 1.228164  [ 5005/ 6576]
loss: 0.682262  [ 5505/ 6576]
loss: 0.492340  [ 6005/ 6576]
loss: 1.344578  [ 6505/ 6576]
Epoch 3
-------------------------------
loss: 0.748557  [    5/ 6576]
loss: 0.68

In [17]:
torch.save(model.state_dict(), "./model_resnet")

# Finishing training on handmade dataset

In [4]:
from torchvision.models import resnet50, ResNet50_Weights
model_resnet_retrain = resnet50(weights=ResNet50_Weights.DEFAULT)

num_features = model_resnet_retrain.fc.in_features
num_classes = 8

model_resnet_retrain.fc = nn.Sequential(
    nn.Linear(num_features, 1000),  # New layer with 1000 outputs (matches pre-trained model)
    nn.ReLU(),                       
    nn.Linear(1000, num_classes)    # Final layer with 8 outputs for your classification task
)

model_resnet_retrain = model_resnet_retrain.cuda()

model_resnet_retrain.load_state_dict(torch.load("./model_resnet"))

weights = ResNet50_Weights.DEFAULT
preprocess = weights.transforms()

In [5]:
path_pair_img = '../data/photoshoptest_handmade'
dataset = datasets.ImageFolder(root=path_pair_img, transform=preprocess)

batch_size = 5
train_dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [7]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_resnet_retrain.parameters(), lr=1e-3)

In [8]:
epochs = 20
torch.cuda.empty_cache()
for i in range(1):
    for t in range(epochs):
        print(f"Epoch {t+1}\n-------------------------------")
        train(train_dataloader, model_resnet_retrain, loss_fn, optimizer)
    print("Done!")


Epoch 1
-------------------------------
loss: 1.026977  [    5/  121]
Epoch 2
-------------------------------
loss: 0.621275  [    5/  121]
Epoch 3
-------------------------------
loss: 0.424157  [    5/  121]
Epoch 4
-------------------------------
loss: 0.255032  [    5/  121]
Epoch 5
-------------------------------
loss: 0.235204  [    5/  121]
Epoch 6
-------------------------------
loss: 0.168232  [    5/  121]
Epoch 7
-------------------------------
loss: 0.028000  [    5/  121]
Epoch 8
-------------------------------
loss: 0.013707  [    5/  121]
Epoch 9
-------------------------------
loss: 0.029799  [    5/  121]
Epoch 10
-------------------------------
loss: 0.001812  [    5/  121]
Epoch 11
-------------------------------
loss: 0.794239  [    5/  121]
Epoch 12
-------------------------------
loss: 0.001180  [    5/  121]
Epoch 13
-------------------------------
loss: 0.050332  [    5/  121]
Epoch 14
-------------------------------
loss: 0.047060  [    5/  121]
Epoch 15
------

In [10]:
torch.save(model_resnet_retrain.state_dict(), "./model_resnet_retrain")